In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import re

2022-02-25 07:54:17.534680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-25 07:54:17.534719: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Define Model


In [2]:
num_classes = 2
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(num_classes, activation='softmax'))

# not using the first layer for training
model.layers[0].trainable = False

2022-02-25 07:55:10.993064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 07:55:10.993404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-25 07:55:10.993483: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-25 07:55:10.993538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-25 07:55:10.993584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

# Compile Model

In [3]:
model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

# Image Augmentation

In [4]:
# data augmentation for training images
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True, 
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
# Speicy no augmentation that will be used for validation data
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Take image directly from director and apply augmentation

In [5]:
#define image size height and width
image_size = 224

#Prepare training image
train_generator = data_generator_with_aug.flow_from_directory(
                                        directory='/home/tohid/Desktop/Danaxa/Project/train',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')
#Prepare validation image
validation_generator = data_generator_no_aug.flow_from_directory(
                                        directory='/home/tohid/Desktop/Danaxa/Project/val',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

Found 1200 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


# Fit Model

In [6]:
fit_stats = model.fit_generator(train_generator,
                                       steps_per_epoch=60,
                                       epochs = 4,
                                       validation_data=validation_generator,
                                       validation_steps=1)

/tmp/ipykernel_7341/1442894033.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_stats = model.fit_generator(train_generator,


Epoch 1/4
60/60 [==============================] - 30s 465ms/step - loss: 0.1820 - accuracy: 0.9317 - val_loss: 0.1018 - val_accuracy: 0.9688
Epoch 2/4
60/60 [==============================] - 29s 475ms/step - loss: 0.0647 - accuracy: 0.9750 - val_loss: 0.0810 - val_accuracy: 0.9375
Epoch 3/4
60/60 [==============================] - 28s 457ms/step - loss: 0.0510 - accuracy: 0.9850 - val_loss: 0.0247 - val_accuracy: 1.0000
Epoch 4/4
60/60 [==============================] - 27s 454ms/step - loss: 0.0465 - accuracy: 0.9867 - val_loss: 0.0249 - val_accuracy: 1.0000


# Testing with test image

In [8]:
#collect test data from directory and process
test_generator = data_generator_no_aug.flow_from_directory(
    directory='/home/tohid/Desktop/Danaxa/Project/test',
    target_size=(image_size, image_size),
    batch_size= 10,
    class_mode=None,
    shuffle=False
)

# Predict from generator (returns probabilities)
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=test_generator.filenames
# Data frame

real_class = []
for file in filenames:
  if re.search("with_mask", file):
    real_class.append(1.0)
  else:
    real_class.append(0.0)

predicted_class = cl[:,0]

results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "pred_class":predicted_class, "real_class":real_class})

Found 200 images belonging to 2 classes.


/tmp/ipykernel_7341/3247160529.py:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)


20/20 [==============================] - 9s 435ms/step


In [10]:
res = sum(1 for x,y in zip(real_class,predicted_class) if x == y) / len(real_class)
print("Accuracy :", res)

Accuracy : 0.995


# Save Model

In [11]:
model.save('maskDetectionModel.h5')

/home/tohid/anaconda3/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
